## BlackSheep Cookbook Exploration

The Black Sheep Analysis allows researchers to find trends in abnormal protein enrichment among patients in CPTAC datasets. In this Cookbook, we will go through the steps needed to perform a full Black Sheep Analysis.

### Step 1a: Import Dependencies
First, import the necessary dependencies and load cptac data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cptac
import binarization_functions as bf
import blackSheepCPTACmoduleCopy as blsh
import gseapy as gp
from gseapy.plot import barplot, heatmap, dotplot

Welcome to cptac, a python package for disseminating cancer
proteogenomics data. To view available datasets, enter
'cptac.list_data()'. Extensive tutorials are available at
https://github.com/PayneLab/cptac/tree/master/doc

******
Version: 0.4.5
******


## Step 1b: Load Data and Choose Omics Table
For this analysis, we will be looking at results across the proteomics, transcriptomics, and phosphoproteomics tables.

In [2]:
co = cptac.Colon()
proteomics = co.get_proteomics()
transcriptomics = co.get_transcriptomics()
clinical = co.get_clinical()

Checking that data files are up-to-date...
100% [..................................................................................] 487 / 487
Data check complete.
colon data version: Most recent release

Loading clinical data...
Loading miRNA data...
Loading mutation data...
Loading mutation_binary data...
Loading phosphoproteomics_normal data...
Loading phosphoproteomics_tumor data...
Loading proteomics_normal data...
Loading proteomics_tumor data...
Loading transcriptomics data...


## Step 2: Determine what attributes you would like to A/B test. 
For this analysis, we will iteratively go through the various columns in the clinical dataset, to determine if any of them have trends within them for protein enrichment.

In [10]:
#Create a copy of the original Clinical DataFrame and drop irrelevant columns.
annotations = pd.DataFrame(clinical.copy())
set(annotations)

{'Age',
 'CEA',
 'Gender',
 'Lymphatic_Invasion',
 'Mucinous',
 'Mutation_Phenotype',
 'Patient_ID',
 'Perineural_Invasion',
 'Polyps_History',
 'Polyps_Present',
 'Proteomic_subtype',
 'Sample_Tumor_Normal',
 'Stage',
 'Subsite',
 'Synchronous_Tumors',
 'Transcriptomic_subtype',
 'Tumor.Status',
 'Vascular_Invasion',
 'Vital.Status',
 'mutation_rate',
 'pathalogy_N_stage',
 'pathalogy_T_stage'}

In [ ]:
annotations = annotations.drop(['Patient_ID'], axis=1)

In [29]:
already_binary_columns = ['Mutation_Phenotype', 'Tumor.Status', 
                          'Vital.Status', 'Polyps_Present', 
                          'Polyps_History', 'Synchronous_Tumors', 
                          'Perineural_Invasion', 'Lymphatic_Invasion', 
                          'Vascular_Invasion', 'Mucinous', 'Gender', 
                          'Sample_Tumor_Normal']

#should pathalogy_T_stage be pathology_T_stage??
columns_2_binarize = ['Age', 'Subsite', 
                      'pathalogy_T_stage',
                      'pathalogy_N_stage', 
                      'Stage', 'CEA', 
                      'Transcriptomic_subtype', 
                      'Proteomic_subtype', 
                      'mutation_rate']

In [30]:
for col in columns_2_binarize:
    print(annotations[col].value_counts())
    print('\n')

902     3
708     2
904     2
658     2
643     2
695     2
656     2
883     2
768     2
737     2
840     2
623     1
1055    1
859     1
1052    1
610     1
1020    1
881     1
798     1
940     1
912     1
810     1
778     1
792     1
813     1
892     1
838     1
1038    1
575     1
480     1
       ..
947     1
442     1
894     1
805     1
684     1
734     1
589     1
853     1
876     1
469     1
757     1
620     1
895     1
758     1
723     1
848     1
689     1
996     1
667     1
948     1
755     1
865     1
423     1
930     1
652     1
697     1
660     1
1065    1
832     1
771     1
Name: Age, Length: 96, dtype: int64


Sigmoid Colon       38
Ascending Colon     27
Cecum               23
Descending Colon    10
Hepatix Flexure      7
Splenic Flexure      3
Tranverse Colon      1
Name: Subsite, dtype: int64


T3     80
T2     16
T4a    12
T4b     2
Name: pathalogy_T_stage, dtype: int64


N0     56
N1b    18
N1a    14
N2a     9
N2b     7
N1      6
Name: pathalogy_N_sta

## Step 2a: Binarize column values

In [ ]:
for val in annotations['Age']: #why on earth are these all strings?
    print(type(val))

In [39]:
'''annotations['Age'] = bf.binarizeCutOff(clinical, 'Age', 
                                       730, '2 years or older', 
                                       'Younger than 2 years')'''

"annotations['Age'] = bf.binarizeCutOff(clinical, 'Age', \n                                       730, '2 years or older', \n                                       'Younger than 2 years')"

In [37]:
subsite_map = {'Sigmoid Colon':'Sigmoid_Colon', 
               'Ascending Colon':'Other_site', 
               'Cecum ':'Other_site', 
               'Descending Colon':'Other_site', 
               'Hepatix Flexure':'Other_site', 
               'Splenic Flexure':'Other_site', 
               'Tranverse Colon':'Other_site'}

annotations['Subsite'] = bf.binarizeCategorical(clinical, 
                                                'Subsite', 
                                                subsite_map)

In [ ]:
pathalogy_T_stage_map = {'T3':'T3orT2', 'T2':'T3orT2', 
                         'T4a':'T4', 'T4b':'T4'}

annotations['pathalogy_T_stage'] = bf.binarizeCategorical(clinical, 
                                                          'pathalogy_T_stage', 
                                                          pathalogy_T_stage_map)

In [44]:
pathalogy_N_stage_map = {'N0':'N0', 'N1':'N1orN2',
                         'N1a':'N1orN2', 'N1b':'N1orN2', 
                         'N2a':'N1orN2', 'N2b':'N1orN2'}

annotations['pathalogy_N_stage'] = bf.binarizeCategorical(clinical, 
                                                          'pathalogy_N_stage', 
                                                          pathalogy_N_stage_map)

In [46]:
stage_map = {'Stage I':'StageIorII', 
             'Stage II':'StageIorII', 
             'Stage III':'StageIIIorIV', 
             'Stage IV':'StageIIIorIV'}

annotations['Stage'] = bf.binarizeCategorical(clinical, 
                                              'Stage', 
                                              stage_map)

## Step 3: Perform outliers analysis

In [4]:
outliers_prot = blsh.make_outliers_table(proteomics, iqrs=1.5, 
                                         up_or_down='up', 
                                         aggregate=False, 
                                         frac_table=False)

outliers_trans = blsh.make_outliers_table(transcriptomics, iqrs=1.5, 
                                          up_or_down='up', 
                                          aggregate=False, 
                                          frac_table=False)

## Step 4: Wrap your A/B test into the outliers analysis, and create a table
First for proteomics, and then phosphoproteomics.

In [5]:
results_prot = blsh.compare_groups_outliers(outliers_prot, 
                                            annotations)

NameError: name 'annotations' is not defined

In [ ]:
results_trans = blsh.compare_groups_outliers(outliers_trans, 
                                             annotations)

Many of the output values from compare_group_outliers are NaN, so here we will get rid of the NaN values for visualization purposes.

In [ ]:
results_prot = results_prot.dropna(axis=0, how='all')
results_trans = results_trans.dropna(axis=0, how='all')

## Step 5: Visualize these enrichments

In [ ]:
sns.heatmap(results_prot)
plt.show()

In [ ]:
sns.heatmap(results_trans)
plt.show()

In [ ]:
#How can we automate something like this?
results_prot = results_prot.drop(['Proteomics_Tumor_Normal_Other_tumor_enrichment_FDR', 
                                  'Histologic_Grade_FIGO_High_grade_enrichment_FDR',
                                  'Histologic_Grade_FIGO_Low_grade_enrichment_FDR', 
                                  'Myometrial_invasion_Specify_50%_or_more_enrichment_FDR'], 
                                 axis=1)

## Step 6: Determine significant enrichments, and link with cancer drug database.

In [ ]:
print("TESTING FOR PROTEOMICS:")
sig_cols = []
for col in results_prot.columns:
    sig_col = bf.significantEnrichments(results_prot, col, 0.025)
    if sig_col is not None:
        sig_cols.append(sig_col)
    else:
        continue

In [ ]:
for col in sig_cols:
    col_name = col.columns[0]
    gene_name_list = list(col.index)
    enrichment = gp.enrichr(gene_list = gene_name_list, 
                            description=col_name, 
                            gene_sets='KEGG_2019_Human', 
                            outdir='test/enrichr_kegg', #This isn't saving correctly...why is that?
                            cutoff=0.5)
    print(enrichment.res2d)
    barplot(enrichment.res2d, title=col_name)